In [133]:
import pandas as pd
import json
import glob
from functools import reduce

pd.options.display.max_columns = None

In [165]:
#Get the file names from data folder
all_files = glob.glob('data' + '/*.csv.gz')

column_order = ['neighbourhood_cleansed', '2018 nov', '2018 dec', '2019 jan', '2019 feb', '2019 mar',
                '2019 apr', '2019 may', '2019 jun', '2019 jul', '2019 aug', '2019 sep', '2019 oct',
                '2019 nov', '2019 dec', '2020 jan', '2020 feb', '2020 mar', '2020 apr', '2020 may',
                '2020 jun', '2020 jul', '2020 aug', '2020 sep', '2020 oct']

#We only consider private rooms and entire homes as other room types are rare
PR_revenue_dfs = []
PR_price_dfs = []
PR_occupancy_dfs = []
PR_count_dfs = []

Etr_revenue_dfs = []
Etr_price_dfs = []
Etr_occupancy_dfs = []
Etr_count_dfs = []

for filename in all_files: 
    print(filename)
    df = pd.read_csv(filename, compression='gzip')
    #Convert price to int datatype
    df.price.replace('[\$\,]', '', regex=True, inplace = True)
    df['price'] = df['price'].astype(float)
    df['price_per_accommodate'] = df['price'] / df['accommodates']
    #Occupancy of the last 30 days
    df['occupancy'] = 30 - df['availability_30']
    #Revenue of the last 30 days
    df['revenue'] = df['price'] * df['occupancy']
    #seperate dataframe by private room and entire room
    PR_df = df.loc[df['room_type'] == 'Private room']
    Etr_df = df.loc[df['room_type'] == 'Entire home/apt']
    #Average price, occupancy and revenue of each room type by neighbours of the last 30 days
    PR_price = PR_df.groupby('neighbourhood_cleansed')['price_per_accommodate'].mean().round(decimals=2).reset_index(name = filename[14:22])
    Etr_price = PR_df.groupby('neighbourhood_cleansed')['price_per_accommodate'].mean().round(decimals=2).reset_index(name = filename[14:22])
    PR_revenue = PR_df.groupby('neighbourhood_cleansed')['revenue'].mean().round(decimals=2).reset_index(name = filename[14:22])
    Etr_revenue = Etr_df.groupby('neighbourhood_cleansed')['revenue'].mean().round(decimals=2).reset_index(name = filename[14:22])
    PR_occupancy = PR_df.groupby('neighbourhood_cleansed')['occupancy'].mean().round(decimals=2).reset_index(name = filename[14:22])
    Etr_occupancy = Etr_df.groupby('neighbourhood_cleansed')['occupancy'].mean().round(decimals=2).reset_index(name = filename[14:22])
    PR_price_dfs.append(PR_price)
    Etr_price_dfs.append(Etr_price)
    PR_revenue_dfs.append(PR_revenue)
    Etr_revenue_dfs.append(PR_revenue)
    PR_occupancy_dfs.append(PR_occupancy)
    Etr_occupancy_dfs.append(PR_occupancy)
    
PR_price = reduce(lambda left,right: pd.merge(left,right,on=['neighbourhood_cleansed']), PR_price_dfs)
PR_price = PR_price[column_order]

PR_revenue = reduce(lambda left,right: pd.merge(left,right,on=['neighbourhood_cleansed']), PR_revenue_dfs)
PR_revenue = PR_revenue[column_order]

PR_occupancy = reduce(lambda left,right: pd.merge(left,right,on=['neighbourhood_cleansed']), PR_occupancy_dfs)
PR_occupancy = PR_occupancy[column_order]

PR_price.head(10)

data\listings 2018 dec.csv.gz
data\listings 2018 nov.csv.gz
data\listings 2019 apr.csv.gz
data\listings 2019 aug.csv.gz
data\listings 2019 dec.csv.gz
data\listings 2019 feb.csv.gz
data\listings 2019 jan.csv.gz
data\listings 2019 jul.csv.gz
data\listings 2019 jun.csv.gz
data\listings 2019 mar.csv.gz
data\listings 2019 may.csv.gz
data\listings 2019 nov.csv.gz
data\listings 2019 oct.csv.gz
data\listings 2019 sep.csv.gz
data\listings 2020 apr.csv.gz
data\listings 2020 aug.csv.gz
data\listings 2020 feb.csv.gz
data\listings 2020 jan.csv.gz
data\listings 2020 jul.csv.gz
data\listings 2020 jun.csv.gz
data\listings 2020 mar.csv.gz
data\listings 2020 may.csv.gz
data\listings 2020 oct.csv.gz
data\listings 2020 sep.csv.gz


,neighbourhood_cleansed,2018 nov,2018 dec,2019 jan,2019 feb,2019 mar,2019 apr,2019 may,2019 jun,2019 jul,2019 aug,2019 sep,2019 oct,2019 nov,2019 dec,2020 jan,2020 feb,2020 mar,2020 apr,2020 may,2020 jun,2020 jul,2020 aug,2020 sep,2020 oct
0,Agincourt North,30.25,30.26,30.72,30.44,29.99,29.23,28.81,29.30,28.51,27.70,27.34,27.77,26.89,28.85,29.02,29.39,29.09,29.23,32.57,35.85,34.99,35.49,36.16,37.02
1,Agincourt South-Malvern West,33.70,32.23,30.67,29.26,28.12,29.94,29.77,31.78,35.66,35.78,37.22,36.32,34.75,32.54,32.65,33.04,40.27,39.92,41.56,40.91,42.07,40.23,41.79,39.96
2,Alderwood,34.14,38.15,43.05,41.22,36.96,38.29,37.35,35.04,34.00,37.38,31.54,29.05,30.71,29.18,28.77,27.53,28.25,28.28,27.97,28.68,31.62,36.05,38.83,109.61
3,Annex,43.48,44.38,44.43,44.21,45.54,45.50,44.15,44.37,45.56,45.52,44.37,44.85,44.86,43.55,43.26,44.61,44.67,46.09,43.33,42.80,43.23,47.42,47.94,46.75
4,Banbury-Don Mills,56.38,52.01,56.68,55.73,42.74,39.56,41.42,40.47,54.42,50.14,52.60,53.51,58.86,60.94,62.41,61.16,60.03,55.88,59.62,60.20,58.56,46.86,49.84,53.79
5,Bathurst Manor,40.27,36.75,35.57,35.75,36.77,38.29,39.31,39.77,41.65,43.30,39.67,44.76,42.72,40.60,34.85,42.00,40.99,45.78,47.43,45.81,45.70,46.70,48.49,44.83
6,Bay Street Corridor,76.25,76.63,72.32,72.57,84.02,75.36,77.02,77.27,75.54,72.87,74.50,72.40,71.72,68.34,68.52,67.88,69.10,68.76,75.86,77.15,80.12,69.13,69.91,70.40
7,Bayview Village,34.72,35.54,35.26,37.11,38.15,39.21,44.05,43.92,43.31,43.84,43.65,44.30,44.83,44.18,42.81,43.12,42.66,43.89,43.73,44.10,43.61,37.23,38.14,36.92
8,Bayview Woods-Steeles,32.58,32.15,29.65,30.43,29.12,28.55,30.76,32.75,32.38,35.25,34.38,35.07,36.34,36.68,35.56,35.12,36.51,36.50,36.43,35.37,33.29,38.17,38.74,40.55
9,Bedford Park-Nortown,39.85,37.17,36.29,35.80,29.62,28.96,25.33,24.50,37.19,27.20,37.25,36.94,28.30,31.08,37.15,37.17,32.17,28.70,32.31,29.34,33.56,168.00,106.39,106.26


In [179]:
PR_revenue.head()

,neighbourhood_cleansed,2018 nov,2018 dec,2019 jan,2019 feb,2019 mar,2019 apr,2019 may,2019 jun,2019 jul,2019 aug,2019 sep,2019 oct,2019 nov,2019 dec,2020 jan,2020 feb,2020 mar,2020 apr,2020 may,2020 jun,2020 jul,2020 aug,2020 sep,2020 oct
0,Agincourt North,704.11,790.68,774.93,762.04,767.72,706.68,733.90,893.18,637.26,784.76,840.41,789.02,748.49,759.35,798.10,691.92,865.22,832.76,996.30,1007.12,953.72,1015.83,1002.03,913.74
1,Agincourt South-Malvern West,972.71,659.36,608.69,622.27,573.02,675.07,743.02,928.62,910.34,1013.51,1182.68,1033.39,1006.97,932.12,847.03,951.44,1364.65,1271.70,1515.00,1255.68,1346.96,1456.46,1629.08,1425.95
2,Alderwood,302.22,510.40,898.64,853.60,1004.67,1276.86,1040.80,925.71,997.00,1082.57,1014.23,1105.36,1023.00,1018.36,827.82,593.93,761.50,813.50,1043.31,883.59,909.25,995.91,726.44,3129.78
3,Annex,1408.55,1396.41,1409.42,1459.05,1432.85,1474.74,1412.69,1405.07,1475.92,1678.11,1678.03,1587.03,1492.28,1426.17,1382.21,1482.02,1417.82,1474.81,1399.74,1472.13,1535.12,1588.75,1472.00,1480.25
4,Banbury-Don Mills,1943.88,1497.50,1052.72,976.69,1092.05,1334.20,1385.72,1236.35,1135.75,1360.96,1534.83,1348.31,1033.17,2284.88,1866.47,1673.09,1151.82,1165.75,2005.43,1291.21,2205.14,1446.98,1549.07,2003.63


In [167]:
PR_revenue.to_csv('PR_revenue.csv', index = False)

In [172]:
PR_occupancy.to_csv('PR_occupancy.csv', index = False)

In [170]:
PR_occupancy.head()

,neighbourhood_cleansed,2018 nov,2018 dec,2019 jan,2019 feb,2019 mar,2019 apr,2019 may,2019 jun,2019 jul,2019 aug,2019 sep,2019 oct,2019 nov,2019 dec,2020 jan,2020 feb,2020 mar,2020 apr,2020 may,2020 jun,2020 jul,2020 aug,2020 sep,2020 oct
0,Agincourt North,14.04,15.71,14.85,15.41,17.66,16.48,16.55,19.59,14.86,17.82,18.83,17.22,16.58,16.12,17.21,14.65,18.43,17.57,19.27,19.00,21.44,21.55,21.14,19.41
1,Agincourt South-Malvern West,16.46,12.16,11.35,12.50,12.07,12.93,14.35,18.19,16.14,17.04,20.71,18.75,16.94,17.07,16.39,18.51,19.23,16.64,21.38,17.29,18.01,20.44,23.53,21.17
2,Alderwood,8.00,9.20,14.09,12.50,17.92,22.00,19.13,20.86,22.38,22.79,22.92,24.55,22.92,22.45,18.18,14.00,19.06,19.75,24.94,20.24,20.33,21.55,16.56,20.89
3,Annex,20.74,20.32,20.40,20.69,20.04,20.57,19.91,20.03,20.88,22.68,22.99,21.51,20.34,19.63,19.60,20.72,19.89,19.82,19.99,20.87,21.42,21.31,20.77,20.58
4,Banbury-Don Mills,17.04,11.93,11.72,11.54,13.14,18.75,19.00,17.76,16.79,19.88,22.12,18.92,12.79,16.28,18.44,15.47,13.43,14.61,17.71,15.00,19.43,19.48,18.64,21.52


In [171]:
PR_price.to_csv('PR_price.csv', index = False)

In [175]:
df_2020_oct = pd.read_csv('data\listings 2020 oct.csv.gz', compression = 'gzip')
df_2020_oct = df_2020_oct.loc[df_2020_oct['neighbourhood_cleansed'] == 'Alderwood']
df_2020_oct[['listing_url', 'price', 'availability_30', 'availability_60', 'availability_365']].head(50)

,listing_url,price,availability_30,availability_60,availability_365
1903,https://www.airbnb.com/rooms/7256445,$149.00,0,0,0
2101,https://www.airbnb.com/rooms/7966411,$78.00,0,5,125
2214,https://www.airbnb.com/rooms/8400387,$79.00,0,0,0
2780,https://www.airbnb.com/rooms/10922780,$105.00,7,37,342
3344,https://www.airbnb.com/rooms/13224752,$699.00,0,0,22
3481,https://www.airbnb.com/rooms/13477083,$18.00,0,0,0
3897,https://www.airbnb.com/rooms/14202538,$60.00,24,54,359
3939,https://www.airbnb.com/rooms/14282377,$699.00,29,59,89
5279,https://www.airbnb.com/rooms/18199503,$74.00,18,48,76
6797,https://www.airbnb.com/rooms/21261765,$41.00,0,0,299


In [178]:
df_2020_oct = pd.read_csv('data\listings 2020 sep.csv.gz', compression = 'gzip')
df_2020_oct = df_2020_oct.loc[df_2020_oct['neighbourhood_cleansed'] == 'Bedford Park-Nortown']
df_2020_oct[['listing_url', 'price', 'minimum_nights', 'availability_30', 'availability_60', 'availability_365']].head(50)

,listing_url,price,minimum_nights,availability_30,availability_60,availability_365
1359,https://www.airbnb.com/rooms/5944041,$250.00,1,0,0,0
2923,https://www.airbnb.com/rooms/10925910,$65.00,1,0,0,0
3409,https://www.airbnb.com/rooms/12988600,$89.00,2,9,35,339
3868,https://www.airbnb.com/rooms/13863591,$99.00,1,0,0,0
5254,https://www.airbnb.com/rooms/17516509,$600.00,30,0,0,98
6498,https://www.airbnb.com/rooms/20309933,$90.00,4,7,7,7
6755,https://www.airbnb.com/rooms/20800535,$130.00,2,26,56,109
7052,https://www.airbnb.com/rooms/21249409,$35.00,1,0,0,0
7308,https://www.airbnb.com/rooms/21677096,$80.00,3,8,11,11
7694,https://www.airbnb.com/rooms/22401027,$85.00,1,0,0,0
